The following code block installs the simpy library, which is needed to simulate the production, and import other libraries that are necessary.

In [ ]:
!pip install simpy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import simpy
from tqdm.auto import tqdm
import itertools
import concurrent.futures
import time
import random
import json

The following code block defines the simualtion model.

In [ ]:
class ProductionSystem:
    def __init__(
        self,
        env,
        handlers,
        stations,
        component_assignments,
        inventory_allocations,
        reorder_points,
        metrics_dict,
        seed=0,
    ):
        self.env = env
        self.handlers = handlers
        self.stations = stations
        self.metrics_dict = metrics_dict
        self.orders_outstanding = {
            'Station 1': False,
            'Station 2': False,
            'Station 3': False,
            'Station 4': False,
            'Station 5': False,
        }
        self.travel_time_distributions = {
            'Station 1': 5 + (2 + 1.5*handlers.capacity)*np.random.lognormal(sigma=0.2),
            'Station 2': 10 + (2 + 1.5*handlers.capacity)*np.random.lognormal(sigma=0.3),
            'Station 3': 12.5 + (2 + 1.5*handlers.capacity)*np.random.lognormal(sigma=0.4),
            'Station 4': 15 + (2 + 1.5*handlers.capacity)*np.random.lognormal(sigma=0.5),
            'Station 5': 20 + (2 + 1.5*handlers.capacity)*np.random.lognormal(sigma=0.5),
        }
        self.component_assignments = component_assignments
        self.inventory_allocations = inventory_allocations
        self.reorder_points = reorder_points
        self.available_inventory = dict(self.inventory_allocations)
        self.inventory_position = dict(self.available_inventory)

        np.random.seed(seed)

    def process_item(self, vehicle, vehicle_data):
        """Simulate an item moving through five stations."""

        for station in self.stations:
            with self.stations[station].request() as request:
                yield request

                assigned_component = self.component_assignments[station]
                component_variant = vehicle_data[assigned_component]

                if self.available_inventory[component_variant] == 0:
                    self.metrics_dict[f'{assigned_component}_repairs'] += 1
                else:
                    self.available_inventory[component_variant] -= 1
                    self.inventory_position[component_variant] -= 1

                yield self.env.timeout(1)

    def inventory_control(self):
        """Periodically check the level of the gas station tank and call the tank
        truck if the level falls below a threshold."""
        for station in self.stations:
            if not self.orders_outstanding[station]:
                assigned_component = self.component_assignments[station]
                orders = {}
                for component_variant in self.reorder_points:
                    if component_variant.startswith(assigned_component):
                        if self.inventory_position[component_variant] <= self.reorder_points[component_variant]:
                            order_size = self.inventory_allocations[component_variant] - self.available_inventory[component_variant]
                            orders[component_variant] = order_size
                if orders:
                    self.orders_outstanding[station] = True
                    with self.handlers.request() as handler_request:
                        yield handler_request
                        orders = {}
                        for component_variant in self.reorder_points:
                            if component_variant.startswith(assigned_component):
                                if self.inventory_position[component_variant] <= self.reorder_points[component_variant]:
                                    order_size = self.inventory_allocations[component_variant] - self.available_inventory[component_variant]
                                    orders[component_variant] = order_size

                        order_time = self.env.now
                        for ordered_component, ordered_amount in orders.items():
                            self.inventory_position[ordered_component] += ordered_amount

                        yield self.env.timeout(self.travel_time_distributions[station])

                        for ordered_component, ordered_amount in orders.items():
                            self.available_inventory[ordered_component] += ordered_amount
                        self.orders_outstanding[station] = False

def generate_arrivals(env, production_system, vehicle_information):
    """Generate arrivals based on given arrival times."""
    for vehicle, vehicle_data in vehicle_information.items():
        arrival_time = vehicle_data['arrival_time']
        yield env.timeout(arrival_time - env.now)

        env.process(production_system.process_item(vehicle, vehicle_data))
        env.process(production_system.inventory_control())

The following code block defines a function to run the simulation model.

In [ ]:
def run_simulation(
    production_data,
    component_assignments,
    inventory_allocations,
    reorder_points,
    num_handlers=1,
    day_start=1,
    day_end=90,
    space_available=100,
) -> dict:

    assigned_components_list = sorted(list(component_assignments.values()))
    assert assigned_components_list == ['CA', 'CB', 'CC', 'CD', 'CE'], "You need to assign each component to a single station"

    total_component_allocations = {
        'CA': 0,
        'CB': 0,
        'CC': 0,
        'CD': 0,
        'CE': 0,
    }
    for cvariant, callocation in inventory_allocations.items():
        if cvariant.startswith('CA'):
            total_component_allocations['CA'] += callocation
        if cvariant.startswith('CB'):
            total_component_allocations['CB'] += callocation
        if cvariant.startswith('CC'):
            total_component_allocations['CC'] += callocation
        if cvariant.startswith('CD'):
            total_component_allocations['CD'] += callocation
        if cvariant.startswith('CE'):
            total_component_allocations['CE'] += callocation

    for ccomponent, ctotal_allocation in total_component_allocations.items():
        assert ctotal_allocation <= space_available, f'You are allocating more than {space_available} units for variants of component {ccomponent}'

    for cvariant, creorder_point in reorder_points.items():
        assert reorder_points[cvariant] < inventory_allocations[cvariant], f'Reorder point must be less than inventory allocation for {cvariant}'
        assert reorder_points[cvariant] >= 0, f'Reorder point must be greater than or equal to zero for {cvariant}'


    component_variants = production_data.group_by(
        'day'
    ).agg(
        pl.col('CA'),
        pl.col('CB'),
        pl.col('CC'),
        pl.col('CD'),
        pl.col('CE'),
    ).sort(
        'day'
    ).to_pandas().set_index(
        'day'
    ).to_dict(orient='index')

    repair_wage_rate = 45
    tugger_wage_rate = 30

    repair_times = {
        'CA': 10,
        'CB': 8,
        'CC': 12,
        'CD': 4,
        'CE': 8,
    }

    all_metrics = []
    for day in tqdm(range(day_start, day_end + 1), f'Simulating {day_end + 1 -day_start} days'):
        day_vehicle_count = len(component_variants[day]['CA'])
        minutes_available = 960

        arrival_random_numbers = np.random.uniform(
            low=0.4,
            high=0.6,
            size=day_vehicle_count,
        )
        arrival_random_numbers_cumsum = arrival_random_numbers.cumsum()
        arrival_random_numbers_cumsum_normalized = (
            arrival_random_numbers_cumsum/arrival_random_numbers_cumsum.max()
        )
        arrival_times = minutes_available*arrival_random_numbers_cumsum_normalized

        vehicle_info_zip = zip(
            arrival_times,
            component_variants[day]['CA'],
            component_variants[day]['CB'],
            component_variants[day]['CC'],
            component_variants[day]['CD'],
            component_variants[day]['CE'],
        )

        vehicle_information = {}
        for vehicle, (arrival_time, CA, CB, CC, CD, CE) in enumerate(vehicle_info_zip, 1):
            vehicle_information[vehicle] = {
                'arrival_time': float(arrival_time),
                'CA': CA,
                'CB': CB,
                'CC': CC,
                'CD': CD,
                'CE': CE,
            }

        metrics_dict = {
            'CA_repairs': 0,
            'CB_repairs': 0,
            'CC_repairs': 0,
            'CD_repairs': 0,
            'CE_repairs': 0,
        }

        # Setup the simulation environment
        env = simpy.Environment()

        # Create resources for each station (assuming each station can handle one item at a time)
        stations = {f'Station {idx}': simpy.Resource(env, capacity=1) for idx in range(1, 6)}
        handlers = simpy.Resource(env, capacity=num_handlers)

        # Create the production system
        production_system = ProductionSystem(
            env,
            handlers=handlers,
            stations=stations,
            component_assignments=component_assignments,
            inventory_allocations=inventory_allocations,
            reorder_points=reorder_points,
            metrics_dict=metrics_dict,
            seed=day,
        )

        # Generate arrivals based on interarrival times
        env.process(generate_arrivals(env, production_system, vehicle_information))

        # Run the simulation
        env.run()

        run_metrics = dict(metrics_dict)
        run_metrics.update({
            'day': day,
            'num_handlers': num_handlers,
        })
        run_metrics['CA_repair_costs'] = repair_wage_rate*((run_metrics['CA_repairs']*repair_times['CA'])/60)
        run_metrics['CB_repair_costs'] = repair_wage_rate*((run_metrics['CB_repairs']*repair_times['CB'])/60)
        run_metrics['CC_repair_costs'] = repair_wage_rate*((run_metrics['CC_repairs']*repair_times['CC'])/60)
        run_metrics['CD_repair_costs'] = repair_wage_rate*((run_metrics['CD_repairs']*repair_times['CD'])/60)
        run_metrics['CE_repair_costs'] = repair_wage_rate*((run_metrics['CE_repairs']*repair_times['CE'])/60)
        run_metrics['MH_costs'] = (minutes_available/60)*run_metrics['num_handlers']*tugger_wage_rate
        run_metrics['Total_repair_costs'] = (
            run_metrics['CA_repair_costs']
            + run_metrics['CB_repair_costs']
            + run_metrics['CC_repair_costs']
            + run_metrics['CD_repair_costs']
            + run_metrics['CE_repair_costs']
        )
        run_metrics['Total_costs'] = run_metrics['MH_costs'] + run_metrics['Total_repair_costs']

        all_metrics.append(dict(run_metrics))

    return all_metrics

The following code block reads the production data and prints the first five rows.

In [ ]:
data = pl.read_csv('https://raw.githubusercontent.com/nkfreeman/2024_IDA_Hackathon/refs/heads/main/production_data.csv')


In [ ]:
def get_inventory_assignments(max_variants, max_storage):
    total_count= len(data)
    max_num_variants= max_variants+1
    variant_percentages= dict()

    for column in data.columns:
        if(column[0] != "C"):
            continue
        else:
            variants=[0]*max_num_variants
            for entry in data.get_column(column):
                variants[int(entry[2])-1]+=1
        i=0
        sum=0
        while(variants[i] != 0 and i<max_num_variants-1):
            key= column + str(i+1)
            num_inventory= round((variants[i]/total_count*max_storage))
            variant_percentages[key]= num_inventory
            sum+=num_inventory
            if(sum>max_storage):
                variant_percentages[key]-=(sum-max_storage)
            i+=1
    return variant_percentages

In [ ]:
def get_neighbors(variants_initial, step_size=1):
    if step_size % 1 != 0:
        print('ERROR: Step size must be integer value')
        return -1
    if variants_initial is None:
        print('ERROR: variants_initial must be provided')
        return -1

    dimensions = len(variants_initial)

    neighbor_ranges = [
        [value - step_size, value, value + step_size] for value in variants_initial
    ]
    neighbors = list(itertools.product(*neighbor_ranges))

    return neighbors

In [ ]:
optimized_reorder_params = {}
optimized_inventory_params= {}

Enter component name ('CA', 'CB', ...) as a function parameter and it will run through the gradient descent until no further improvements are found. Right now it is verbose, so it prints each point it runs and gives an update for the best values found with each loop. we can turn these off so it isnt so overwhelming on the screen in the future.

also, it runs 10 day simulations, so we will need to change that to whatever we want.

In [ ]:
def evaluate_point(point, component, matching_keys, reorder_points_copy, previous_points, start_day, end_day):
    # Evaluate a single point (simulation)
    invalid_point = False
    for i in range(1, len(matching_keys) + 1):
        if inventory_allocations[(str(component) + str(i))] <= point[i - 1]:
            invalid_point = True
        if point in previous_points:
            invalid_point=True
    if invalid_point:
        return None  # Skip invalid point

    # Simulate the reorder with this point
    for i in range(1, len(point) + 1):
        reorder_points_copy[(str(component) + str(i))] = point[i - 1]

    simulation_metrics = pd.DataFrame(run_simulation(
        production_data=data,
        component_assignments=component_assignments,
        inventory_allocations=inventory_allocations,
        reorder_points=reorder_points_copy,
        num_handlers=1,
        day_start=start_day,
        day_end=end_day,
        space_available=100,
    ))
    current_loop_cost = simulation_metrics['Total_costs'].sum()

    return (point, current_loop_cost)

def gradient_optimize_reorder_param_sgd(component, batch_size=5, max_days=90):
    possible_components = ['CA', 'CB', 'CC', 'CD', 'CE']
    if component not in possible_components:
        print('Error: incorrect component name. Eligible component names are:')
        print(possible_components)
        return -1

    matching_keys = [key for key in reorder_points.keys() if key.startswith(str(component))]
    initial_restock_vals = [reorder_points[(str(component) + str(i))] for i in range(1, len(matching_keys) + 1)]

    # Initial simulation to get starting costs
    simulation_metrics = pd.DataFrame(run_simulation(
        production_data=data,
        component_assignments=component_assignments,
        inventory_allocations=inventory_allocations,
        reorder_points=reorder_points,
        num_handlers=1,
        day_start=1,
        day_end=50,  # Start with the first 10 days
        space_available=100,
    ))

    total_cost_current = simulation_metrics['Total_costs'].sum()
    total_cost_best = total_cost_current
    print(f"Initial total cost: {total_cost_best}")

    previous_points = [tuple(initial_restock_vals)]
    best_point = tuple(initial_restock_vals)

    improvement_found = True
    iteration_count = 0  # Track number of iterations

    # Variables to manage day blocks (start and end)
    start_day = 1
    end_day = 50

    while improvement_found:
        improvement_found = False
        # Get all possible neighbors (full set of points)
        all_points = get_neighbors(variants_initial=initial_restock_vals)

        if(len(all_points) > 50):
            batch_size = round(0.01 * len(all_points))
        else:
            batch_size=len(all_points)

        # Adjust the batch size if it exceeds the number of points
        if batch_size > len(all_points):
            mini_batch_points = all_points  # Use all points if batch size exceeds available points
            print(f"Batch size {batch_size} exceeds available points. Evaluating all {len(mini_batch_points)} points.")
        else:
            mini_batch_points = random.sample(all_points, batch_size)

        best_loop_cost = total_cost_current

        print(f"\nIteration {iteration_count + 1}: Evaluating {len(mini_batch_points)} points (days {start_day} to {end_day})...")

        start_time = time.time()  # Track time for each iteration

        # Create a copy of reorder_points to pass into the parallel evaluation
        reorder_points_copy = reorder_points.copy()

        # Using ThreadPoolExecutor to evaluate points in parallel
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_to_point = {
                executor.submit(evaluate_point, point, component, matching_keys, reorder_points_copy.copy(), previous_points, start_day, end_day): point
                for point in mini_batch_points
            }

            # Process results as they are completed
            valid_results = []
            for future in concurrent.futures.as_completed(future_to_point):
                result = future.result()
                if result is not None:
                    valid_results.append(result)

        # Log progress update
        print(f"Valid points evaluated: {len(valid_results)} / {len(mini_batch_points)}")

        # Find the best point from the valid results
        for point, current_loop_cost in valid_results:
            if current_loop_cost < best_loop_cost:
                best_loop_cost = current_loop_cost
                best_point = point

        previous_points = mini_batch_points

        # Update total cost if we find a better configuration
        if best_loop_cost < total_cost_current:
            total_cost_current = best_loop_cost

        if total_cost_current < total_cost_best:
            improvement_found = True
            total_cost_best = total_cost_current

            print(f"\nImprovement found! New best total cost: {total_cost_best}")
            print(f"Best parameters: {best_point}")

            initial_restock_vals = list(best_point)
        else:
            print(f"No improvement in this iteration. Best total cost remains: {total_cost_best}")

        iteration_count += 1

        # Log iteration completion with time taken
        end_time = time.time()
        print(f"Iteration {iteration_count} completed in {end_time - start_time:.2f} seconds.")
        print('--------------------------------------------')

        # Update day blocks for the next iteration
        # start_day += 10
        # end_day += 10
        # if start_day > max_days:
        #     start_day = 1
        #     end_day = 10

    # Final best cost and parameters
    print(f"\nOptimization complete. Lowest total cost: {total_cost_best}")
    print(f"Best parameters: {best_point}")

    for i in range(1, len(matching_keys) + 1):
        optimized_reorder_params[(str(component) + str(i))] = best_point[i-1]
    print(optimized_reorder_params)


def evaluate_point_inventory(point, component, matching_keys, inventory_allocations_copy, previous_points, start_day, end_day):
    # Evaluate a single point (simulation)
    invalid_point = False
    for i in range(1, len(matching_keys) + 1):
        if point[i - 1] < reorder_points[(str(component) + str(i))]:
            invalid_point = True
        if point in previous_points:
            invalid_point = True
        if sum(point) > 100:
            invalid_point = True
    if invalid_point:
        return None  # Skip invalid point

    # Simulate the inventory allocations with this point
    for i in range(1, len(point) + 1):
        inventory_allocations_copy[(str(component) + str(i))] = point[i - 1]

    simulation_metrics = pd.DataFrame(run_simulation(
        production_data=data,
        component_assignments=component_assignments,
        inventory_allocations=inventory_allocations_copy,
        reorder_points=reorder_points,
        num_handlers=1,
        day_start=start_day,
        day_end=end_day,
        space_available=100,
    ))
    current_loop_cost = simulation_metrics['Total_costs'].sum()

    return (point, current_loop_cost)

def gradient_optimize_inventory_allocations_param_sgd(component, batch_size=5, max_days=90):
    possible_components = ['CA', 'CB', 'CC', 'CD', 'CE']
    if component not in possible_components:
        print('Error: incorrect component name. Eligible component names are:')
        print(possible_components)
        return -1

    matching_keys = [key for key in inventory_allocations.keys() if key.startswith(str(component))]
    initial_restock_vals = [inventory_allocations[(str(component) + str(i))] for i in range(1, len(matching_keys) + 1)]

    # Initial simulation to get starting costs
    simulation_metrics = pd.DataFrame(run_simulation(
        production_data=data,
        component_assignments=component_assignments,
        inventory_allocations=inventory_allocations,
        reorder_points=reorder_points,
        num_handlers=1,
        day_start=1,
        day_end=50,  # Start with the first 10 days
        space_available=100,
    ))

    total_cost_current = simulation_metrics['Total_costs'].sum()
    total_cost_best = total_cost_current
    print(f"Initial total cost: {total_cost_best}")

    previous_points = [tuple(initial_restock_vals)]
    best_point = tuple(initial_restock_vals)

    improvement_found = True
    iteration_count = 0  # Track number of iterations

    # Variables to manage day blocks (start and end)
    start_day = 1
    end_day = 50

    while improvement_found:
        improvement_found = False
        # Get all possible neighbors (full set of points)
        all_points = get_neighbors(variants_initial=initial_restock_vals)
        if(len(all_points) > 50):
            batch_size = round(0.01 * len(all_points))
        else:
            batch_size=len(all_points)
        # Adjust the batch size if it exceeds the number of points
        if batch_size > len(all_points):
            mini_batch_points = all_points  # Use all points if batch size exceeds available points
            print(f"Batch size {batch_size} exceeds available points. Evaluating all {len(mini_batch_points)} points.")
        else:
            mini_batch_points = random.sample(all_points, batch_size)

        best_loop_cost = total_cost_current

        print(f"\nIteration {iteration_count + 1}: Evaluating {len(mini_batch_points)} points (days {start_day} to {end_day})...")

        start_time = time.time()  # Track time for each iteration

        # Create a copy of inventory_allocations to pass into the parallel evaluation
        inventory_allocations_copy = inventory_allocations.copy()

        # Using ThreadPoolExecutor to evaluate points in parallel
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_to_point = {
                executor.submit(evaluate_point_inventory, point, component, matching_keys, inventory_allocations_copy.copy(), previous_points, start_day, end_day): point
                for point in mini_batch_points
            }

            # Process results as they are completed
            valid_results = []
            for future in concurrent.futures.as_completed(future_to_point):
                result = future.result()
                if result is not None:
                    valid_results.append(result)

        # Log progress update
        print(f"Valid points evaluated: {len(valid_results)} / {len(mini_batch_points)}")

        # Find the best point from the valid results
        for point, current_loop_cost in valid_results:
            if current_loop_cost < best_loop_cost:
                best_loop_cost = current_loop_cost
                best_point = point

        previous_points = mini_batch_points

        # Update total cost if we find a better configuration
        if best_loop_cost < total_cost_current:
            total_cost_current = best_loop_cost

        if total_cost_current < total_cost_best:
            improvement_found = True
            total_cost_best = total_cost_current

            print(f"\nImprovement found! New best total cost: {total_cost_best}")
            print(f"Best parameters: {best_point}")

            initial_restock_vals = list(best_point)
        else:
            print(f"No improvement in this iteration. Best total cost remains: {total_cost_best}")

        iteration_count += 1

        # Log iteration completion with time taken
        end_time = time.time()
        print(f"Iteration {iteration_count} completed in {end_time - start_time:.2f} seconds.")
        print('--------------------------------------------')

        # Update day blocks for the next iteration
        # start_day += 10
        # end_day += 10
        # if start_day > max_days:
        #     start_day = 1
        #     end_day = 10

    # Final best cost and parameters
    print(f"\nOptimization complete. Lowest total cost: {total_cost_best}")
    print(f"Best parameters: {best_point}")

    for i in range(1, len(matching_keys) + 1):
        optimized_inventory_params[(str(component) + str(i))] = best_point[i-1]




In [ ]:

def brute_force_line_order(component_assignments):
  keys = list(component_assignments.keys())
  values = list(component_assignments.values())
  permutations = itertools.permutations(values)
  min_loop_cost=float('inf')
  for perm in permutations:
    current_assignment = dict(zip(keys,perm))
    simulation_metrics = pd.DataFrame(run_simulation(
            production_data=data,
            component_assignments=current_assignment,
            inventory_allocations=inventory_allocations,
            reorder_points=reorder_points,
            num_handlers=2,
            day_start=1,
            day_end=90,
            space_available=100,
        ))
    current_loop_cost = simulation_metrics['Total_costs'].sum()
    if current_loop_cost<min_loop_cost:
      min_loop_cost=current_loop_cost
  print(min_loop_cost)
  return min_loop_cost
#brute_force_line_order(component_assignments)

In [ ]:
def random_reorder_variation(component):
    possible_components = ['CA', 'CB', 'CC', 'CD', 'CE']

    # Validate the component name
    if component not in possible_components:
        print('Error: incorrect component name. Eligible component names are:')
        print(possible_components)
        return -1

    # Get keys in inventory_allocations that start with the component name
    matching_keys = [key for key in inventory_allocations.keys() if key.startswith(component)]

    # If no matching keys, return an error
    if not matching_keys:
        print(f'Error: No matching keys found for component {component}')
        return -1

    # Get the initial inventory values for the matching keys
    initial_vals = [reorder_points[key] for key in matching_keys]

    # Apply random variation to each value, while ensuring it doesn't exceed the initial value
    random_variation = [
        max(0, val + random.randint(-5, 5)) if val >= 5 else val  # Ensure the variation doesn't exceed the initial value
        for val in initial_vals
    ]

    return random_variation


In [ ]:
def brute_force_handlers():
    num_handlers = [1,2,3,4,5]
    totals = []
    avgs = []

    for handlers in num_handlers:
        simulation_metrics = run_simulation(
        production_data=data,
        component_assignments=component_assignments,
        inventory_allocations=inventory_allocations,
        reorder_points=reorder_points,
        num_handlers=handlers,
        day_start=1,
        day_end=90,
        space_available=100,
    )
        simulation_metrics = pd.DataFrame(simulation_metrics)
        totals.append(simulation_metrics['Total_costs'].sum())
        avgs.append(simulation_metrics['Total_costs'].mean())
        print('number of handlers : ' + str(handlers))
        print('Total: ' + str(totals[-1]))
        print('Average: ' + str(avgs[-1]))
    return totals.index(min(totals))

The following code block shows a sample solution

In [ ]:
#gradient_optimize_reorder_param("CD")

The following code block simulates the solution over the 90 days of production data

In [ ]:
def evaluate_kpis(simulation_metrics) :
  simulation_metrics = pd.DataFrame(simulation_metrics)
  columns_to_include = ['day', 'MH_costs', 'Total_repair_costs', 'Total_costs']

  fig, ax = plt.subplots(1, 1, figsize=(10, 4))
  simulation_metrics[columns_to_include].set_index(
      'day'
  ).plot(
      ax=ax
  )
  ax.spines[['right', 'top']].set_visible(False)
  ax.legend(bbox_to_anchor=(1.01, 1.01))
  plt.show()
  print('Total cost over ' + str(simulation_metrics['day'].iloc[-1]) + ' days: $' + str(simulation_metrics['Total_costs'].sum()))
  print('Average cost over ' + str(simulation_metrics['day'].iloc[-1]) + ' days: $' + str(simulation_metrics['Total_costs'].mean()))

## **MAIN**

In [ ]:
# step 1, optimize line order
component_assignments = {
    'Station 1': 'CA',
    'Station 2': 'CC',
    'Station 3': 'CE',
    'Station 4': 'CD',
    'Station 5': 'CB',
}

# get initial values based on data distribution
alloc= get_inventory_assignments(9,100)

inventory_allocations = {
    'CA1': alloc.get("CA1"), 'CA2': alloc.get("CA2"), 'CA3': alloc.get("CA3"), 'CA4': alloc.get("CA4"), 'CA5': alloc.get("CA5"), 'CA6': alloc.get("CA6"), 'CA7': alloc.get("CA7"), 'CA8': alloc.get("CA8"),
    'CB1': alloc.get("CB1"), 'CB2': alloc.get("CB2"), 'CB3': alloc.get("CB3"),
    'CC1': alloc.get("CC1"), 'CC2': alloc.get("CC2"), 'CC3': alloc.get("CC3"), 'CC4': alloc.get("CC4"), 'CC5': alloc.get("CC5"), 'CC6': alloc.get("CC6"), 'CC7': alloc.get("CC7"),
    'CD1': alloc.get("CD1"), 'CD2': alloc.get("CD2"), 'CD3': alloc.get("CD3"), 'CD4': alloc.get("CD4"), 'CD5': alloc.get("CD5"), 'CD6': alloc.get("CD6"),
    'CE1': alloc.get("CE1"), 'CE2': alloc.get("CE2"), 'CE3': alloc.get("CE3"), 'CE4': alloc.get("CE4"), 'CE5': alloc.get("CE5"), 'CE6': alloc.get("CE6"), 'CE7': alloc.get("CE7"), 'CE8': alloc.get("CE8"), 'CE9': alloc.get("CE9"),
}


reorder_points = {
    'CA1': int(inventory_allocations["CA1"]/2), 'CA2': int(inventory_allocations["CA2"]/2), 'CA3': int(inventory_allocations["CA3"]/2), 'CA4': int(inventory_allocations["CA4"]/2), 'CA5': int(inventory_allocations["CA5"]/2), 'CA6': int(inventory_allocations['CA6']/2), 'CA7': int(inventory_allocations['CA7']/2), 'CA8': int(inventory_allocations['CA8']/2),
    'CB1': int(inventory_allocations['CB1']/2), 'CB2': int(inventory_allocations['CB2']/2), 'CB3': int(inventory_allocations['CB3']/2),
    'CC1': int(inventory_allocations['CC1']/2), 'CC2': int(inventory_allocations['CC2']/2), 'CC3': int(inventory_allocations['CC3']/2), 'CC4': int(inventory_allocations['CC4']/2), 'CC5': int(inventory_allocations['CC5']/2), 'CC6': int(inventory_allocations['CC6']/2), 'CC7': int(inventory_allocations['CC7']/2),
    'CD1': int(inventory_allocations['CD1']/2), 'CD2': int(inventory_allocations['CD2']/2), 'CD3': int(inventory_allocations['CD3']/2), 'CD4': int(inventory_allocations['CD4']/2), 'CD5': int(inventory_allocations['CC5']/2), 'CD6': int(inventory_allocations['CD6']/2),
    'CE1': int(inventory_allocations['CE1']/2), 'CE2': int(inventory_allocations['CE2']/2), 'CE3': int(inventory_allocations['CE3']/2), 'CE4': int(inventory_allocations['CE4']/2), 'CE5': int(inventory_allocations['CE5']/2), 'CE6': int(inventory_allocations['CE6']/2), 'CE7': int(inventory_allocations['CE7']/2), 'CE8': int(inventory_allocations['CE8']/2), 'CE9': int(inventory_allocations['CE9']/2),
}




#step 2, gradient optimization on inventory
components = ['CB','CA','CC','CD','CE']
for component in components:

    gradient_optimize_inventory_allocations_param_sgd(component)
    print(component + ' done')
    inventory_allocations.update(optimized_inventory_params)
# inventory_allocations = optimized_inventory_params

NameError: name 'get_inventory_assignments' is not defined

In [ ]:
#step 3, gradient optimization on reorder
for component in components:
    gradient_optimize_reorder_param_sgd(component)
    print(component + ' done')
    reorder_points.update(optimized_reorder_params)
#reorder_points = optimized_reorder_params

#step 4, brute force handler number
num_handlers = [1,2,3,4,5]
best_handlers = num_handlers[brute_force_handlers()]

#print best total cost
simulation_metrics = pd.DataFrame(run_simulation(
        production_data=data,
        component_assignments=component_assignments,
        inventory_allocations=inventory_allocations,
        reorder_points=reorder_points,
        num_handlers=best_handlers,
        day_start=1,
        day_end=90,
        space_available=100,
    ))

print('======================')
print('======================')

print('======================')
print('FINAL RESULTS')
print(simulation_metrics['Total_costs'].sum())
print(simulation_metrics['Total_costs'].mean())


all_dicts = [component_assignments, inventory_allocations, reorder_points]

# Save as JSON file
with open('optimized_params.json', 'w') as json_file:
    json.dump(all_dicts, json_file, indent=4)  # Use indent=4 for pretty printing

print('Dictionaries saved to JSON file')


NameError: name 'components' is not defined

In [ ]:
def print_temp_data(reorder_points_temp):
  print("Reorder Points Temp: ",reorder_points_temp)
  print("Inventory Allocations: ",inventory_allocations)
  print("Component Assignments: ",component_assignments)



In [ ]:
def print_data():
  print("Reorder Points Temp: ",reorder_points)
  print("Inventory Allocations: ",inventory_allocations)
  print("Component Assignments: ",component_assignments)

In [ ]:
def reset_values(component_assignments_best, inventory_allocations_best, reorder_points_best):
    # Rearrange component_assignments
    stations = list(component_assignments_best.keys())
    random.shuffle(stations)
    new_component_assignments = {station: component_assignments_best[station] for station in stations}

    # Reset inventory allocations while maintaining row sums
    inventory_row_keys = [['CA1', 'CA2', 'CA3', 'CA4', 'CA5', 'CA6', 'CA7', 'CA8'],
                           ['CB1', 'CB2', 'CB3'],
                           ['CC1', 'CC2', 'CC3', 'CC4', 'CC5', 'CC6', 'CC7'],
                           ['CD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6'],
                           ['CE1', 'CE2', 'CE3', 'CE4', 'CE5', 'CE6', 'CE7', 'CE8', 'CE9']]

    new_inventory_allocations = {}
    for row_keys in inventory_row_keys:
        total = 100
        row_allocations = [random.randint(1, total) for _ in range(len(row_keys) - 1)]
        row_allocations.append(total - sum(row_allocations))  # Ensure the sum equals 100
        random.shuffle(row_allocations)
        for key, value in zip(row_keys, row_allocations):
            new_inventory_allocations[key] = value

    # Reset reorder points based on new inventory allocations
    new_reorder_points = {}
    for key in new_inventory_allocations.keys():
        new_reorder_points[key] = random.randint(1, new_inventory_allocations[key] - 1)

    return new_component_assignments, new_inventory_allocations, new_reorder_points

In [ ]:
def brute_force_line_order(component_assignments):
  keys = list(component_assignments.keys())
  values = list(component_assignments.values())
  permutations = sorted(itertools.permutations(values),reverse=True)
  min_loop_cost=float('inf')
  counter=0
  for perm in permutations:
    counter+=1
    current_assignment = dict(zip(keys,perm))
    simulation_metrics = pd.DataFrame(run_simulation(
            production_data=data,
            component_assignments=current_assignment,
            inventory_allocations=inventory_allocations,
            reorder_points=reorder_points,
            num_handlers=2,
            day_start=1,
            day_end=30,
            space_available=100,
        ))

    current_loop_cost = simulation_metrics['Total_costs'].sum()
    if current_loop_cost<min_loop_cost:
      min_loop_cost=current_loop_cost
      evaluate_kpis(simulation_metrics)
      print_data()
    print("Iteration: ",counter)
  print(min_loop_cost)
  return min_loop_cost

In [ ]:
def brute_force(component_assignments, inventory_allocations, reorder_points):
    min_loop_cost = 6922.05
    component_assignments_best = component_assignments.copy()
    inventory_allocations_best = inventory_allocations.copy()
    reorder_points_best = reorder_points.copy()
    print_data()
    num_handlers_temp = 2
    iteration_counter = 0
    reset_limit = 500000

    while True:
        iteration_counter += 1
        component_assignments_temp = component_assignments_best.copy()
        inventory_allocations_temp = inventory_allocations_best.copy()
        reorder_points_temp = reorder_points_best.copy()

        case = random.randint(1, 4)
        if case == 1:
            stations = list(component_assignments_temp.keys())
            station1, station2 = random.sample(stations, 2)
            component_assignments_temp[station1], component_assignments_temp[station2] = (
                component_assignments_temp[station2], component_assignments_temp[station1]
            )
        elif case == 2:
            inventory_allocations_temp = inventory_allocations.copy()
            def swap_within_row(row_keys):
                for _ in range(random.randint(1, len(row_keys))):
                    key1, key2 = random.sample(row_keys, 2)
                    inventory_allocations_temp[key1], inventory_allocations_temp[key2] = inventory_allocations_temp[key2], inventory_allocations_temp[key1]
                    reorder_points_temp[key1], reorder_points_temp[key2] = reorder_points_temp[key2], reorder_points_temp[key1]

            swap_row = random.randint(1, 5)
            if swap_row == 1:
                swap_within_row(['CA1', 'CA2', 'CA3', 'CA4', 'CA5', 'CA6', 'CA7', 'CA8'])
            elif swap_row == 2:
                swap_within_row(['CB1', 'CB2', 'CB3'])
            elif swap_row == 3:
                swap_within_row(['CC1', 'CC2', 'CC3', 'CC4', 'CC5', 'CC6', 'CC7'])
            elif swap_row == 4:
                swap_within_row(['CD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6'])
            else:
                swap_within_row(['CE1', 'CE2', 'CE3', 'CE4', 'CE5', 'CE6', 'CE7', 'CE8', 'CE9'])

        elif case == 3:
            random_key = random.choice(list(reorder_points_temp.keys()))
            inventory_allocation = inventory_allocations_temp[random_key]
            new_reorder_point = int(inventory_allocation * random.uniform(0.2, 0.8))
            new_reorder_point = min(new_reorder_point, inventory_allocation - 1)
            new_reorder_point = max(new_reorder_point, 0)
            reorder_points_temp[random_key] = new_reorder_point
            if new_reorder_point >= inventory_allocation:
                new_reorder_point = inventory_allocation - 1
            if new_reorder_point < 0:
                new_reorder_point = 0
            reorder_points_temp[random_key] = new_reorder_point


        else:
            num_handlers_temp = random.randint(1, 3)

        simulation_metrics = pd.DataFrame(run_simulation(
            production_data=data,
            component_assignments=component_assignments_temp,
            inventory_allocations=inventory_allocations_temp,
            reorder_points=reorder_points_temp,
            num_handlers=num_handlers_temp,
            day_start=1,
            day_end=10,
            space_available=100,
        ))
        current_loop_cost = simulation_metrics['Total_costs'].sum()

        if current_loop_cost < min_loop_cost:
            min_loop_cost = current_loop_cost
            evaluate_kpis(simulation_metrics)
            print_temp_data(reorder_points_temp)
            print("Num of Workers: ", num_handlers_temp)
            print("Total cost over 100 days: ", current_loop_cost * 10)
            component_assignments_best = component_assignments_temp
            inventory_allocations_best = inventory_allocations_temp
            reorder_points_best = reorder_points_temp
            print(iteration_counter)
            iteration_counter = 0  # Reset counter when a new minimum is found

        if iteration_counter >= reset_limit:
            component_assignments_best, inventory_allocations_best, reorder_points_best = reset_values(
                component_assignments_best, inventory_allocations_best, reorder_points_best
            )
            iteration_counter = 0  # Reset counter after values are reset

Current Solution:
reorder_points =  {
   'CA1': 11, 'CA2': 7, 'CA3': 2, 'CA4': 10, 'CA5': 4, 'CA6': 10, 'CA7': 7, 'CA8': 10,
   'CB1': 7, 'CB2': 40, 'CB3': 16,
   'CC1': 1, 'CC2': 16, 'CC3': 4, 'CC4': 11, 'CC5': 6, 'CC6': 11, 'CC7': 13,
   'CD1': 3, 'CD2': 12, 'CD3': 26, 'CD4': 1, 'CD5': 15, 'CD6': 8,
   'CE1': 0, 'CE2': 2, 'CE3': 7, 'CE4': 1, 'CE5': 1, 'CE6': 12, 'CE7': 8, 'CE8': 15, 'CE9': 18}




inventory_allocations =  {'CA1': 15, 'CA2': 12, 'CA3': 4, 'CA4': 14, 'CA5': 6, 'CA6': 15, 'CA7': 10, 'CA8': 15,
                        'CB1': 11, 'CB2': 64, 'CB3': 22,
                        'CC1': 2, 'CC2': 23, 'CC3': 7, 'CC4': 15, 'CC5': 10, 'CC6': 20, 'CC7': 17,
                        'CD1': 5, 'CD2': 16, 'CD3': 38, 'CD4': 2, 'CD5': 22, 'CD6': 13,
                        'CE1': 2, 'CE2': 4, 'CE3': 11, 'CE4': 2, 'CE5': 2, 'CE6': 18, 'CE7': 12, 'CE8': 19, 'CE9': 25}




component_assignments =  {'Station 1': 'CA',
                        'Station 2': 'CC',
                        'Station 3': 'CE',
                        'Station 4': 'CD',
                        'Station 5': 'CB'}

One Worker
Total cost over 90 days: $69220.5
Average cost over 90 days: $769.1166666666667
